<a href="https://colab.research.google.com/github/bernhardtandy/ProjectsMLAI/blob/main/HW1_AndyBernhardt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projects in Machine Learning and AI Homework 1**
##### **Andy Bernhardt**
##### **bernha@rpi.edu**

## Task 1: Problem Statement

## Task 2: Dataset and Exploratory Data Analysis

## Task 3: Implementation of Logistic Regression

## Task 4: Comparison of Optimized Logistic Regression Algorithms